# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

vacay_df=pd.read_csv('output_data/clean_city_data.csv').reset_index()
vacay_df.drop(["level_0","Unnamed: 0", "index"], axis=1, inplace=True)
vacay_df.head()

,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Kapaa,22.0752,-159.3190,61.00,82,1,10.36,US,02/06/21 04:21
1,Forsytheganj,52.6333,29.7333,9.28,92,83,7.31,BY,02/06/21 04:24
2,Saint-Philippe,-21.3585,55.7679,87.80,62,40,11.50,RE,02/06/21 04:21
3,Kloulklubed,7.0419,134.2556,77.00,94,90,10.36,PW,02/06/21 04:22
4,Palmer,42.1584,-72.3287,23.00,93,40,10.36,US,02/06/21 04:24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = vacay_df[["Latitude", "Longitude"]].astype(float)
humidity=vacay_df["Humidity (%)"].astype(float)
locations

,Latitude,Longitude
0,22.0752,-159.3190
1,52.6333,29.7333
2,-21.3585,55.7679
3,7.0419,134.2556
4,42.1584,-72.3287
...,...,...
571,58.8053,-122.7002
572,-35.4444,-60.8831
573,9.2500,0.7833
574,53.9000,11.4667


In [4]:
#Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN",center=(2.0, -40.0), zoom_level=3)

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,max_intensity=100,point_radius=3)

#add layer
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather_df=vacay_df[(vacay_df["Max Temperature (F)"]<=84) & 
                        (vacay_df["Max Temperature (F)"]>=72) & 
                        (vacay_df["Wind Speed (mph)"]<10) & 
                        (vacay_df["Cloudiness (%)"]<=10)].reset_index()
ideal_weather_df.to_csv("output_data/ideal_weather_data.csv")
ideal_weather_df.head()

,index,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,25,Hermanus,-34.4187,19.2345,77.00,67,0,5.01,ZA,02/06/21 04:21
1,35,Poum,-20.2333,164.0167,80.65,70,6,6.44,NC,02/06/21 04:22
2,43,Bredasdorp,-34.5322,20.0403,78.80,53,0,4.61,ZA,02/06/21 04:21
3,73,Bahawalpur,29.4000,71.6833,78.39,17,0,7.70,PK,02/06/21 04:24
4,77,Acapulco de Juárez,16.8634,-99.8901,73.74,80,10,3.24,MX,02/06/21 04:20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#hotel_name=[]
#hotel_df=pd.DataFrame({"Hotel Name":hotel_name})
ideal_weather_df["Hotel Name"]=""
hotel_df=ideal_weather_df
hotel_df.head()

,index,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
0,25,Hermanus,-34.4187,19.2345,77.00,67,0,5.01,ZA,02/06/21 04:21,
1,35,Poum,-20.2333,164.0167,80.65,70,6,6.44,NC,02/06/21 04:22,
2,43,Bredasdorp,-34.5322,20.0403,78.80,53,0,4.61,ZA,02/06/21 04:21,
3,73,Bahawalpur,29.4000,71.6833,78.39,17,0,7.70,PK,02/06/21 04:24,
4,77,Acapulco de Juárez,16.8634,-99.8901,73.74,80,10,3.24,MX,02/06/21 04:20,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "22.2855,114.1577",  # philadelphia coords
    "radius":5000,
    "type": "lodging",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    lati=hotel_df.loc[index,"Latitude"]
    longi=hotel_df.loc[index,"Longitude"]
    lt_lng=f"{lati},{longi}"
    params["location"]=lt_lng
    hotel_data=requests.get(base_url,params=params).json()
    
    try:
        hotel_df.loc[index,"Hotel Name"]=hotel_data['results'][0]['name']
    except IndexError:
        hotel_df.loc[index, "Hotel Name"]="NaN"

hotel_df_cpy=hotel_df.loc[hotel_df["Hotel Name"]!="NaN"]
hotel_df.to_csv("output_data/hotel_data.csv")
hotel_df_cpy

,index,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
0,25,Hermanus,-34.4187,19.2345,77.00,67,0,5.01,ZA,02/06/21 04:21,Misty Waves Boutique Hotel
1,35,Poum,-20.2333,164.0167,80.65,70,6,6.44,NC,02/06/21 04:22,Gîte kajeon
2,43,Bredasdorp,-34.5322,20.0403,78.80,53,0,4.61,ZA,02/06/21 04:21,Bredasdorp Country Manor
3,73,Bahawalpur,29.4000,71.6833,78.39,17,0,7.70,PK,02/06/21 04:24,5th Avenue Hotel
4,77,Acapulco de Juárez,16.8634,-99.8901,73.74,80,10,3.24,MX,02/06/21 04:20,HS Hotsson Smart Acapulco
5,81,Verāval,20.9000,70.3667,83.32,29,0,6.82,IN,02/06/21 04:24,Lords Inn Somnath
6,114,Plettenberg Bay,-34.0527,23.3716,75.85,61,0,6.73,ZA,02/06/21 04:22,Thanda Vista B&B
7,141,Hong Kong,22.2855,114.1577,73.99,65,0,1.01,HK,02/06/21 04:18,Mini Hotel Central
8,154,Umluj,25.0213,37.2685,72.00,44,0,9.48,SA,02/06/21 04:24,Moon light Furnished Units
9,197,Ulladulla,-35.3500,150.4667,75.99,88,0,5.75,AU,02/06/21 04:24,Sandpiper Motel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_cpy.iterrows()]
hotel_locations = hotel_df_cpy[["Latitude", "Longitude"]]

# Add marker layer ontop of heat map
markers=gmaps.marker_layer(hotel_locations,info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))